
# Environment Setup



In [ ]:
!pip install -q cohere pandas numpy tqdm umap-learn hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 55.6 MB/s eta 0:00:00


In [ ]:
import os
import cohere
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
from google.colab import userdata
co = cohere.Client(userdata.get("COHERE_API_KEY"))

In [ ]:
response = co.embed(
    model="embed-english-v3.0",
    texts=["test sentence"],
    input_type="search_document"
)

print(len(response.embeddings[0]))

1024


# Load Cleaned (Filtered) Data

In [ ]:
df = pd.read_csv("Embedding Table.csv")
df.shape
df.columns

Index(['EID', 'Year', 'Embedding Text'], dtype='object')

# Generate Embeddings

In [ ]:
texts = df["Embedding Text"].astype(str).tolist()

In [ ]:
import time

#trial and attempt one of embedding (I only need the time variable as it will afftec the second and also will be used as evidence)

BATCH_SIZE = 48          # smaller batch
SLEEP_TIME = 1.2         # seconds between calls

all_embeddings = []

for i in tqdm(range(0, len(texts), BATCH_SIZE)):
    batch = texts[i:i + BATCH_SIZE]

    try:
        response = co.embed(
            model="embed-english-v3.0",
            texts=batch,
            input_type="search_document"
        )
        all_embeddings.extend(response.embeddings)
        time.sleep(SLEEP_TIME)

    except Exception as e:
        print(f"Error at batch {i}: {e}")
        break

In [ ]:
start_idx = len(all_embeddings)  # continue from where it stopped

#continuation of trial and attempt one of embedding (not important however have not deleted due to possible errors)

for i in tqdm(range(start_idx, len(texts), BATCH_SIZE)):
    batch = texts[i:i + BATCH_SIZE]

    try:
        response = co.embed(
            model="embed-english-v3.0",
            texts=batch,
            input_type="search_document"
        )
        all_embeddings.extend(response.embeddings)
        time.sleep(SLEEP_TIME)

    except Exception as e:
        print(f"Error at batch {i}: {e}")
        print("Sleeping 30s before retry...")
        time.sleep(30)   # wait longer if rate-limited
        continue  # try the same batch again

 11%|█         | 46/414 [01:13<09:51,  1.61s/it]

Error at batch 3648: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': '5dfb08a65c5b0f38c93397f80d84e857', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '63', 'date': 'Sun, 21 Dec 2025 11:50:48 GMT', 'x-envoy-upstream-service-time': '11', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'transfer-encoding': 'chunked'}, status_code: 429, body: {'id': '9c4d8c00-1f34-431d-b47e-819a83d9fc62', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Sleeping 30s before retry...


 24%|██▍       | 100/414 [03:12<08:13,  1.57s/it]

Error at batch 6240: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': '187e812beef5e08dba4fdde7d1ae8655', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '61', 'date': 'Sun, 21 Dec 2025 11:52:46 GMT', 'x-envoy-upstream-service-time': '15', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'transfer-encoding': 'chunked'}, status_code: 429, body: {'id': '9e30a531-af37-4759-9dd0-da457cbc8238', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Sleeping 30s before retry...


 37%|███▋      | 152/414 [05:24<07:52,  1.80s/it]

...

 78%|███████▊  | 322/414 [11:13<02:21,  1.54s/it]

Error at batch 16896: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': '882f7e0f2ad394ca536ed0a07e18f365', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '61', 'date': 'Sun, 21 Dec 2025 12:00:47 GMT', 'x-envoy-upstream-service-time': '11', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'transfer-encoding': 'chunked'}, status_code: 429, body: {'id': '7c276049-af36-435b-b44a-20898fc76703', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Sleeping 30s before retry...


 92%|█████████▏| 381/414 [13:12<00:50,  1.52s/it]

Error at batch 19728: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': 'c12365251c51c49ef7f7675e7f8bd7f3', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '61', 'date': 'Sun, 21 Dec 2025 12:02:47 GMT', 'x-envoy-upstream-service-time': '11', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'transfer-encoding': 'chunked'}, status_code: 429, body: {'id': 'e7381b77-c22c-4a3a-8881-651b54d6acc4', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Sleeping 30s before retry...


100%|██████████| 414/414 [14:31<00:00,  2.11s/it]


In [ ]:
BATCH_SIZE = 96
SLEEP_TIME = 1.2
MAX_RETRIES = 5
SAVE_FILE = "papers_with_embeddings_partial.pkl"

# Load previous progress if exists
if os.path.exists(SAVE_FILE):
    df_saved = pd.read_pickle(SAVE_FILE)
    all_embeddings = df_saved["embedding"].tolist()
    print(f"Resuming from {len(all_embeddings)} embeddings...")
else:
    all_embeddings = []
    print("Starting fresh embedding...")

start_idx = len(all_embeddings)

# Safe embedding loop
for i in tqdm(range(start_idx, len(texts), BATCH_SIZE)): #like we learned in AP Com, start. end. interval.
    batch = texts[i:i + BATCH_SIZE]
    retries = 0

    while retries < MAX_RETRIES:
        try:
            response = co.embed(
                model="embed-english-v3.0",
                texts=batch,  #pretty sure texts is reffering to another variable
                input_type="search_document"
            )

            # Add embeddings
            all_embeddings.extend(response.embeddings)

            # Save in-place (overwrites same file)
            df_partial = pd.DataFrame({
                "EID": df["EID"][:len(all_embeddings)],
                "Year": df["Year"][:len(all_embeddings)],
                "Embedding Text": df["Embedding Text"][:len(all_embeddings)],
                "embedding": all_embeddings
            })
            df_partial.to_pickle(SAVE_FILE)

            time.sleep(SLEEP_TIME)
            break

        except Exception as e:
            retries += 1
            wait_time = 10 * retries
            print(f"Error at batch {i}, retry {retries}/{MAX_RETRIES}: {e}")
            print(f"Sleeping {wait_time}s before retry...")
            time.sleep(wait_time)
    else:
        print(f"Batch {i} failed after {MAX_RETRIES} retries. Skipping this batch...")

Starting fresh embedding...


  7%|▋         | 15/222 [00:28<06:25,  1.86s/it]

Error at batch 1440, retry 1/5: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': '85f34d830d0d639bf52298f6ae75c386', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '84', 'date': 'Sun, 21 Dec 2025 12:16:41 GMT', 'x-envoy-upstream-service-time': '12', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'transfer-encoding': 'chunked'}, status_code: 429, body: {'id': 'ccb18360-a654-43d3-b90f-bd42c1edc7dc', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Sleeping 10s before retry...
Error at batch 1440, retry 2/5: headers: {'access-contro

 13%|█▎        | 29/222 [01:25<06:41,  2.08s/it]

Error at batch 2784, retry 1/5: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': 'cc2607d7b960629093b94072a9196741', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '82', 'date': 'Sun, 21 Dec 2025 12:17:38 GMT', 'x-envoy-upstream-service-time': '74', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'transfer-encoding': 'chunked'}, status_code: 429, body: {'id': '2506d214-8c34-414b-bb72-39ce62d94aa5', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Sleeping 10s before retry...
Error at batch 2784, retry 2/5: headers: {'access-contro

...

 91%|█████████ | 202/222 [13:34<00:55,  2.77s/it]

Error at batch 19392, retry 1/5: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': 'c35cdc859e07627af2063ecf4bb6c90f', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '81', 'date': 'Sun, 21 Dec 2025 12:29:47 GMT', 'x-envoy-upstream-service-time': '14', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'transfer-encoding': 'chunked'}, status_code: 429, body: {'id': 'eeea3473-5c4c-43af-a476-b7e951d0f868', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Sleeping 10s before retry...
Error at batch 19392, retry 2/5: headers: {'access-cont

 98%|█████████▊| 217/222 [14:46<00:14,  2.96s/it]

Error at batch 20832, retry 1/5: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': 'f8fab17865997ceb87f5b3af7d444182', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '84', 'date': 'Sun, 21 Dec 2025 12:30:59 GMT', 'x-envoy-upstream-service-time': '19', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'transfer-encoding': 'chunked'}, status_code: 429, body: {'id': '87029e15-6695-4f14-86f1-ca595cbe6a62', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Sleeping 10s before retry...


100%|██████████| 222/222 [15:11<00:00,  4.10s/it]


In [ ]:
df_final = pd.read_pickle(SAVE_FILE)
len(df_final["embedding"][0])

NameError: name 'SAVE_FILE' is not defined

In [ ]:
df_final.to_pickle("papers_with_embeddings.pkl")

In [ ]:
#Testing whether or not the file actually embedded
import pandas as pd

df_final = pd.read_pickle("papers_with_embeddings.pkl")  # or partial file if needed

# Pick a specific row
idx = 1000
print("Text:", df_final["Embedding Text"][idx])
print("Embedding vector length:", len(df_final["embedding"][idx]))
print("First 10 numbers of embedding:", df_final["embedding"][idx][:10])

Text: Title: Developing a conceptual framework for flexible surge capacity based on complexity and collaborative theoretical frameworks. Abstract: https://www.scopus.com/record/display.url?eid=2-s2.0-85131458009&origin=resultslist
Embedding vector length: 1024
First 10 numbers of embedding: [0.010917664, -0.011459351, -0.039367676, -0.10144043, -0.0067710876, -0.009750366, -0.06390381, 0.034698486, -0.040863037, -0.0011720657]


In this step, test embeddings are complete and saved as papers_with_embeddings.pkl.
Please do not re-run the embedding step as it is expensive and a very long process! - Ted

# Store Embeddings

In [ ]:
import pandas as pd

# Load the final pickle file
df_final = pd.read_pickle("papers_with_embeddings.pkl")

In [ ]:
df_final["embedding_str"] = df_final["embedding"].apply(lambda x: ",".join(map(str, x)))

In [ ]:
df_final.to_csv("Embedding Table with Embeddings.csv", index=False)

In [ ]:
#alternative for downloading the .pkl file from the files sidebar.
from google.colab import files

# Replace with your file name
files.download("papers_with_embeddings.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Reload Embeddings

In [ ]:
import pandas as pd

df = pd.read_pickle("papers_with_embeddings.pkl")
df.shape

(21276, 4)

In [ ]:
type(df["embedding"][0]), len(df["embedding"][0])

(list, 1024)

In [ ]:
df["embedding"].isnull().sum()

np.int64(0)

# Topic Modeling / Clustering

In [ ]:
!pip install -q hdbscan

In [ ]:
import numpy as np
import hdbscan

In [ ]:
X = np.vstack(df["embedding"].values) #this makes it so that it takes the lists from df and only gets the numbers
X.shape

(21276, 1024)

In [ ]:
import umap

umap_cluster = umap.UMAP(
    n_neighbors=30,
    n_components=10,     # this UMAP is not used for visualization but to reduce dimensions for clustering
    min_dist=0.0,
    metric="cosine",
    random_state=42
)

X_cluster = umap_cluster.fit_transform(X)

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [ ]:
import hdbscan

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=15,
    min_samples=5,
    metric="euclidean",
    cluster_selection_method="eom"
)

df["topic"] = clusterer.fit_predict(X_cluster)

In [ ]:
n_clusters = len(set(df["topic"])) - (1 if -1 in df["topic"].values else 0)
noise_ratio = (df["topic"] == -1).mean()

n_clusters, noise_ratio


(226, np.float64(0.4291690167324685))

In [ ]:
df.to_pickle("research_with_topics.pkl") #I saved this immediately

In [ ]:
df[df["topic"] == 0]["Embedding Text"].head(3).tolist() #test clusters and it does work.

['Title: Burkholderia pseudomallei and melioidosis. Abstract: https://www.scopus.com/record/display.url?eid=2-s2.0-85173948545&origin=resultslist',
 'Title: Antibiotic susceptibility of clinical burkholderia pseudomallei isolates in northeast thailand from 2015 to 2018 and the genomic characterization of β-lactam-resistant isolates. Abstract: https://www.scopus.com/record/display.url?eid=2-s2.0-85105098806&origin=resultslist',
 'Title: Burkholderia pseudomallei multi-centre study to establish EUCAST MIC and zone diameter distributions and epidemiological cut-off values. Abstract: https://www.scopus.com/record/display.url?eid=2-s2.0-85089090234&origin=resultslist']

In [ ]:
df_no_embed = df.drop(columns=["embedding"])
df_no_embed.to_csv("research_with_topics.csv", index=False)

# Global Dimensionality Reduction (UMAP)

In [ ]:
import numpy as np

X = np.vstack(df["embedding"].values)
X.shape

(21276, 1024)

In [ ]:
import umap

umap_model = umap.UMAP(
    n_neighbors=30,        # balances global vs local structure
    n_components=2,        # 2D research map
    min_dist=0.1,          # keeps clusters visible but not collapsed
    metric="cosine",       # correct for embeddings
    random_state=42
)

In [ ]:
X_umap = umap_model.fit_transform(X)
X_umap.shape

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


(21276, 2)

# Attach UMAP Coordinates

In [ ]:
df["umap_x"] = X_umap[:, 0]
df["umap_y"] = X_umap[:, 1]

In [ ]:
df[["umap_x", "umap_y"]].head() #checking

,umap_x,umap_y
0,0.443239,10.263789
1,-0.959806,8.074731
2,-0.133030,6.775570
3,-0.153236,6.852670
4,1.505602,8.039685


# Save Research Map Data

In [ ]:
df.to_pickle("research_with_topics_umap.pkl")

In [ ]:
df.drop(columns=["embedding"]).to_csv(
    "research_map_ready.csv",
    index=False
)

#Topic Modelling Revision (Less Noise)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_pickle("03_Embeddings Table.pkl")

In [ ]:
df.shape
df.columns


Index(['EID', 'Year', 'Embedding Text', 'embedding'], dtype='object')

In [ ]:
X = np.vstack(df["embedding"].values)
X.shape

(21276, 1024)

In [ ]:
import umap

umap_for_cluster = umap.UMAP(
    n_neighbors=15,
    n_components=10,
    min_dist=0.0,
    metric="cosine",
    random_state=42
)

X_umap_cluster = umap_for_cluster.fit_transform(X)

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [ ]:
from hdbscan import HDBSCAN

clusterer_revised = HDBSCAN(
    min_cluster_size=30,
    min_samples=5, #changed from 10 in the original.
    cluster_selection_method="eom"
)

df["topic_revised"] = clusterer_revised.fit_predict(X_umap_cluster)

/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [ ]:
(df["topic_revised"] == -1).mean() # the most that I could do the lower sensitive while still maintaining accuracy.


np.float64(0.36538823087046435)

In [ ]:
(df["topic_revised"] == -1).sum()


np.int64(7774)

In [ ]:
df.columns

Index(['EID', 'Year', 'Embedding Text', 'embedding', 'topic_revised'], dtype='object')

In [ ]:
df.to_csv(
    "04_Topics_for_Clustering_revised.csv",
    index=False
)

In [ ]:
df.to_pickle(
    "04_Topics_for_Clustering_revised.pkl"
)

In [ ]:
import pandas as pd

df_topics = df.copy()  # my current df with topic_revised
df_completed = pd.read_csv("06_Research Map Completed Data Table.csv")

In [ ]:
len(df_topics), len(df_completed)

(21276, 21276)

In [ ]:
(df_topics.index == df_completed.index).mean() #checking whether or not order is the same

np.float64(1.0)

In [ ]:
(df_topics["Embedding Text"] == df_completed["Embedding Text"]).mean()

np.float64(1.0)

In [ ]:
df_completed["topic"] = df_topics["topic_revised"] #in the new table basically replace every old topic numbers with the new topics.

In [ ]:
df_completed["topic"].value_counts().head()

,count
topic,
-1,7774
57,721
120,683
90,269
72,233


In [ ]:
df_completed = df_completed.rename(columns={
    "topic": "topic_revised"
})

In [ ]:
df_completed.to_pickle("06_Revised:Research Map Completed Table.pkl")

In [ ]:
df_completed.to_csv(
    "06_Revised:Research Map Completed Table.csv",
    index=False
)

# Filtering Check

In [ ]:
#Source of papers dropped: Filtering Process, that I applied in the first steps as seen in the excel sheet 01. , as I filtered for Mahidol, Ramathibodi and Sririray Hospital, Certain Yeras, and Article Types.